In [96]:
with open('input.txt', 'r') as f:
    lines = f.read().split('\n\n')

lines = [l.split('\n') for l in lines]

class Monkey():
    def __init__(self, items, operation, x, test, case1, case2):
        self.items = items
        self.operation = operation
        self.x = x
        self.test = test
        self.case1 = case1
        self.case2 = case2
        self.count = 0
    
    def __repr__(self):
        return '[' + ', '.join([str(n) for n in self.items]) + ']'
    
    def __str__(self):
        return '[' + ', '.join([str(n) for n in self.items]) + ']'

    
    def inspect(self):
        assert(self.items)
        self.count += 1
        item = self.items.pop(0)
        worry_level = (self._operate(item))//3
        throw_to = self._test(worry_level)
        return worry_level, throw_to
    
    def _operate(self, input):
        if self.operation == '*' and self.x == 'old':
            return input * input
        elif self.operation == '+' and self.x == 'old':
            return input + input
        elif self.operation == '*':
            return input * self.x
        elif self.operation == '+':
            return input + self.x
    
    def _test(self, worry):
        if (worry % self.test) == 0:
            return self.case1
        else:
            return self.case2
    
    def receive(self, item):
        self.items.append(item)

monkeys = []

for (i, block) in enumerate(lines):
    # first line
    starting_items = block[1].split(':')[1].split(',')
    starting_items = [int(i.strip()) for i in starting_items]

    # second line
    operation_line = block[2].split('=')[1].strip().split(' ')
    operation = operation_line[1]
    x = operation_line[2]
    x = x if not x.isdigit() else int(x)

    # third line
    test = int(block[3].split(' ')[-1])

    # fourth & fifth
    case1 = int(block[4].split(' ')[-1])
    case2 = int(block[5].split(' ')[-1])
    
    monkey = Monkey(starting_items, operation, x, test, case1, case2)
    monkeys.append(monkey)


for _ in range(20):
    for monkey in monkeys:
        while monkey.items:
            worry_lvl, throw_to = monkey.inspect()
            monkeys[throw_to].receive(worry_lvl)

out = 1
counts = sorted([monkey.count for monkey in monkeys])
out = counts[-1] * counts[-2]

print(out)

54036


In [105]:
# for any      x % a = r,
#           f(x) % a = f(r) % a

class EfficientMonkey():
    def __init__(self, items, operation, remainder_idx, x, case1, case2, other_remainders):
        self.items = items
        self.operation = operation
        self.remainder_idx = remainder_idx
        self.x = x
        self.case1 = case1
        self.case2 = case2
        self.other_remainders = other_remainders
        self.count = 0
    
    def __repr__(self):
        return '[' + ', '.join([str(n) for n in self.items]) + ']'
    
    def __str__(self):
        return '[' + ', '.join([str(n) for n in self.items]) + ']'
    
    def inspect(self):
        assert(self.items)
        self.count += 1

        item = self.items.pop(0)
        # operate on remainders
        worry_remainders = self._operate(item)

        # take remainder first
        out = []
        for (i, r) in enumerate(self.other_remainders):
            out.append(worry_remainders[i]%r)

        throw_to = self._test(out)

        return out, throw_to
    
    def _operate(self, remainders):
        for (i, remainder) in enumerate(remainders):
            if self.operation == '*' and self.x == 'old':
                out = remainder * remainder
            elif self.operation == '+' and self.x == 'old':
                out = remainder + remainder
            elif self.operation == '*':
                out = remainder * self.x
            elif self.operation == '+':
                out = remainder + self.x
            remainders[i] = out
        return remainders

    def _test(self, worry_remainders):
        remainder = worry_remainders[self.remainder_idx] 
        if remainder == 0:
            return self.case1
        else:
            return self.case2
    
    def receive(self, item):
        self.items.append(item)

monkeys = []

# get remainders
remainders = []
for (i, block) in enumerate(lines):
    test = int(block[3].split(' ')[-1])
    remainders.append(test)


for (i, block) in enumerate(lines):
    # first line
    starting_items = block[1].split(':')[1].split(',')
    starting_items = [int(i.strip()) for i in starting_items]

    starting_items = [[item%r for r in remainders] for item in starting_items]

    # second line
    operation_line = block[2].split('=')[1].strip().split(' ')
    operation = operation_line[1]
    x = operation_line[2]
    x = x if not x.isdigit() else int(x)

    # third line (already parsed)

    # fourth & fifth
    case1 = int(block[4].split(' ')[-1])
    case2 = int(block[5].split(' ')[-1])
    
    monkey = EfficientMonkey(starting_items, operation, i, x, case1, case2, remainders)
    monkeys.append(monkey)

for _ in range(10000):
    for monkey in monkeys:
        while monkey.items:
            worry_lvl, throw_to = monkey.inspect()
            monkeys[throw_to].receive(worry_lvl)

out = 1
counts = [monkey.count for monkey in monkeys]
print(counts)
counts = sorted(counts)
out = counts[-1] * counts[-2]

assert(out == 13237873355)

[71076, 35485, 88637, 106189, 17918, 109795, 56795, 120569]
